In [43]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

games = [
    {'id': 261550, 'name': 'Mount__Blade_II_Bannerlord'}
]

def store_page_url(game_id):
    return f'https://store.steampowered.com/app/{game_id}/'

driver_path = '/home/snirlugassy/Documents/School/Gorem/HW2/geckodriver'
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')

driver = webdriver.Firefox(options=options, executable_path=driver_path)


/tmp/ipykernel_135737/298493077.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(options=options, executable_path=driver_path)


In [44]:
@dataclass
class SteamGame:
    title: str
    description: str
    developer: str
    publisher: str
    rating: int
    review_count: int
    release_date: int
    features: list[str]
    tags: list[str]
    img: str
    min_sys_req: str
    rec_sys_req: str

In [45]:

driver.get(store_page_url(games[0]['id']))
soup = BeautifulSoup(driver.page_source, 'html')

In [46]:
game_title = soup.find('div', id='appHubAppName').text.strip()
game_title

'Mount & Blade II: Bannerlord'

In [47]:
review_count = int(soup.find('meta', itemprop='reviewCount')['content'])
review_count

138600

In [48]:
rating = int(soup.find('meta', itemprop='ratingValue')['content'])
rating

9

In [49]:
release_date = soup.find('div', class_='release_date').find('div', class_='date').text.strip()
release_date

'30 Mar, 2020'

In [50]:
game_description = soup.find('div', class_='game_description_snippet').text.strip()
game_description

'The horns sound, the ravens gather. An empire is torn by civil war. Beyond its borders, new kingdoms rise. Gird on your sword, don your armour, summon your followers and ride forth to win glory on the battlefields of Calradia. Establish your hegemony and create a new world out of the ashes of the old.'

In [51]:
developer = soup.find('div', id='developers_list').a.text.strip()
developer

'TaleWorlds Entertainment'

In [52]:
dev_info = {}
for dev_row in soup.find(class_='glance_ctn').find_all(class_='dev_row'):
    dev_info[dev_row.find(class_='subtitle').text.replace(':','')] = dev_row.find(class_='summary').text.strip()
dev_info

{'Developer': 'TaleWorlds Entertainment',
 'Publisher': 'TaleWorlds Entertainment'}

In [53]:
tags = []
for tag in soup.find(class_='glance_tags popular_tags').find_all('a'):
    if 'display: none' not in tag['style']:
        tags.append(tag.text.strip())
tags

['Medieval', 'Strategy', 'Open World', 'RPG', 'War']

In [54]:
game_image = soup.find('img', class_='game_header_image_full')['src']
game_image

'https://cdn.cloudflare.steamstatic.com/steam/apps/261550/header.jpg?t=1626026727'

In [55]:
features = []
for spec in soup.find('div', id='category_block').find_all(class_='game_area_details_specs_ctn'):
    features.append(spec.find(class_='label').text.strip())
features

['Single-player',
 'Online PvP',
 'Partial Controller Support',
 'Steam Cloud',
 'Remote Play on Tablet']

In [56]:
min_sys_req = []
min_sys_req_soup = soup.find('div', class_='game_area_sys_req').find('div', class_='game_area_sys_req_leftCol')
for req in min_sys_req_soup.find('ul', class_='bb_ul').find_all('li'):
    min_sys_req.append(req.text.strip())
min_sys_req

['Requires a 64-bit processor and operating system',
 'OS: Windows 7 (64-bit only)',
 'Processor: Intel® Core™ i3-8100 / AMD Ryzen™ 3 1200',
 'Memory: 6 GB RAM',
 'Graphics: Intel® UHD Graphics 630 / NVIDIA® GeForce® GTX 660 2GB / AMD Radeon™ HD 7850 2GB',
 'Storage: 60 GB available space',
 'Additional Notes: Integrated GPUs require additional 2GB of system RAM. These estimates may change during final release.']

In [57]:
rec_sys_req = []
rec_sys_req_soup = soup.find('div', class_='game_area_sys_req').find('div', class_='game_area_sys_req_rightCol')
for req in rec_sys_req_soup.find('ul', class_='bb_ul').find_all('li'):
    rec_sys_req.append(req.text.strip())
rec_sys_req

['Requires a 64-bit processor and operating system',
 'OS: Windows 10 (64-bit only)',
 'Processor: Intel® Core™ i5-9600K / AMD Ryzen™ 5 3600X',
 'Memory: 8 GB RAM',
 'Graphics: NVIDIA® GeForce® GTX 1060 3GB / AMD Radeon™ RX 580',
 'Storage: 60 GB available space',
 'Additional Notes: These estimates may change during final release.']

In [58]:
steam_game = SteamGame(
    title=game_title,
    description=game_description,
    developer=dev_info['Developer'],
    publisher=dev_info['Publisher'],
    rating=rating,
    review_count=review_count,
    release_date=release_date,
    features=features,
    tags=tags,
    img=game_image,
    min_sys_req=min_sys_req,
    rec_sys_req=rec_sys_req
)

steam_game

SteamGame(title='Mount & Blade II: Bannerlord', description='The horns sound, the ravens gather. An empire is torn by civil war. Beyond its borders, new kingdoms rise. Gird on your sword, don your armour, summon your followers and ride forth to win glory on the battlefields of Calradia. Establish your hegemony and create a new world out of the ashes of the old.', developer='TaleWorlds Entertainment', publisher='TaleWorlds Entertainment', rating=9, review_count=138600, release_date='30 Mar, 2020', features=['Single-player', 'Online PvP', 'Partial Controller Support', 'Steam Cloud', 'Remote Play on Tablet'], tags=['Medieval', 'Strategy', 'Open World', 'RPG', 'War'], img='https://cdn.cloudflare.steamstatic.com/steam/apps/261550/header.jpg?t=1626026727', min_sys_req=['Requires a 64-bit processor and operating system', 'OS: Windows 7 (64-bit only)', 'Processor: Intel® Core™ i3-8100 / AMD Ryzen™ 3 1200', 'Memory: 6 GB RAM', 'Graphics: Intel® UHD Graphics 630 / NVIDIA® GeForce® GTX 660 2GB / 

In [2]:
import time

def reviews_page_url(game_id):
    return f'https://steamcommunity.com/app/{game_id}/reviews/?p=1&browsefilter=toprated'


driver.get(reviews_page_url(games[0]['id']))


In [3]:
try:
    btn = driver.find_element_by_id('age_gate_btn_continue')
    btn.click()
    time.sleep(3)
except:
    pass

/tmp/ipykernel_135737/91235584.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_id('age_gate_btn_continue')


In [4]:
REVIEW_LIMIT = 200

review_cards = driver.find_elements_by_class_name('apphub_Card')

while len(review_cards) < REVIEW_LIMIT:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    review_cards = driver.find_elements_by_class_name('apphub_Card')

/tmp/ipykernel_135737/2070099955.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review_cards = driver.find_elements_by_class_name('apphub_Card')
/tmp/ipykernel_135737/2070099955.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review_cards = driver.find_elements_by_class_name('apphub_Card')


In [7]:
soup = BeautifulSoup(driver.page_source)
cards = soup.find_all('div', class_='apphub_Card')[:REVIEW_LIMIT]
len(cards)

200

In [33]:
from dataclasses import dataclass

@dataclass
class SteamReview:
    text: str
    helpful: str
    rewards: int
    hrs_on_record: float
    

In [35]:
import re

reviews = []

for card in cards:
    recommendation = card.find('div', class_='reviewInfo').find(class_='title').text.strip()

    helpful = card.find('div', class_='found_helpful')

    for br in helpful.find_all('br'):
        br.replaceWith(', ')

    helpful_text = []
    for _child in helpful.children:
        if isinstance(_child, str):
            helpful_text.append(_child)

    helpful_text = ''.join(helpful_text).strip()

    review_total_rewards = int(card.find(class_='review_award_aggregated').text.strip())

    review_text = []
    for _child in card.find('div', class_='apphub_CardTextContent').children:
        if isinstance(_child, str):
            review_text.append(_child)

    review_text = ''.join(review_text).strip()

    hrs_on_record = card.find('div', class_='reviewInfo').find(class_='hours').text.strip()
    hrs_on_record = re.search('[0-9.]+', hrs_on_record)
    if hrs_on_record:
        hrs_on_record = float(hrs_on_record.group())

    reviews.append(SteamReview(
        text=review_text,
        helpful=helpful_text,
        rewards=review_total_rewards,
        hrs_on_record=hrs_on_record
    ))

In [36]:
reviews

[SteamReview(text='A lord refused to nominate me for a castle after I successfully captured it so I switched sides, tracked down his wife, and chopped her head off. 10/10', helpful='2,049 people found this review helpful, 2,436 people found this review funny', rewards=20, hrs_on_record=247.3),
 SteamReview(text="One of the best War games I've played plus I used Voiceattack for voice commands. I felt like a field general lol wife thought I was nuts.", helpful='312 people found this review helpful, 266 people found this review funny', rewards=47, hrs_on_record=443.0),
 SteamReview(text="Has a long way to go. Needs more quest content, more depth, some fixes, added character interactions and dialogue and alot more.It's still fun, it's a modern warband after all. But it could be much more.", helpful='2,072 people found this review helpful, 24 people found this review funny', rewards=2, hrs_on_record=68.4),
 SteamReview(text='This is my first review on steam ever. I felt it was kind of my du